In [1]:
import lifetimes as lt
import cmdstanpy as cmd
import pandas as pd
import numpy as np

/home/braydentang/anaconda3/envs/cltv-proj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_data = pd.read_table(
    "data/CDNOW_sample.txt",
    delim_whitespace=True,
    header=None, 
)

In [3]:
raw_data

,0,1,2,3,4
0,4,1,19970101,2,29.33
1,4,1,19970118,2,29.73
2,4,1,19970802,1,14.96
3,4,1,19971212,2,26.48
4,21,2,19970101,3,63.34
...,...,...,...,...,...
6914,23556,2356,19970726,3,45.74
6915,23556,2356,19970927,3,31.47
6916,23556,2356,19980103,2,28.98
6917,23556,2356,19980607,2,28.98


In [4]:
raw_data.columns = ["customer_id", "customer_index", "date", "quantity", "amount"]
new_data = raw_data.assign(
    new_date=lambda x: x["date"].astype("str"),
    date=lambda x: pd.to_datetime([f"{row[:4]}-{row[4:6]}-{row[6:]}" for row in x["new_date"]])
).drop(columns=["new_date"])

In [5]:
new_data

,customer_id,customer_index,date,quantity,amount
0,4,1,1997-01-01,2,29.33
1,4,1,1997-01-18,2,29.73
2,4,1,1997-08-02,1,14.96
3,4,1,1997-12-12,2,26.48
4,21,2,1997-01-01,3,63.34
...,...,...,...,...,...
6914,23556,2356,1997-07-26,3,45.74
6915,23556,2356,1997-09-27,3,31.47
6916,23556,2356,1998-01-03,2,28.98
6917,23556,2356,1998-06-07,2,28.98


In [6]:
train = new_data.query('date < "1998-01-01"')
test = new_data.query('date >= "1998-01-01"')

train

,customer_id,customer_index,date,quantity,amount
0,4,1,1997-01-01,2,29.33
1,4,1,1997-01-18,2,29.73
2,4,1,1997-08-02,1,14.96
3,4,1,1997-12-12,2,26.48
4,21,2,1997-01-01,3,63.34
...,...,...,...,...,...
6912,23556,2356,1997-06-10,2,26.73
6913,23556,2356,1997-07-19,2,29.33
6914,23556,2356,1997-07-26,3,45.74
6915,23556,2356,1997-09-27,3,31.47


In [7]:
train["amount_indicator"] = train.groupby("customer_id", as_index=False)["date"].transform(lambda x: x == x.min())
train_amount_filtered = train.loc[train["amount_indicator"] == 0, ["customer_id", "amount"]]
train_amount_filtered = train_amount_filtered.groupby("customer_id", as_index=False).agg("mean")
train_amount_filtered

,customer_id,amount
0,4,23.723333
1,21,11.770000
2,111,75.347778
3,112,11.770000
4,114,25.550000
...,...,...
1035,23500,16.192500
1036,23506,13.970000
1037,23507,30.637500
1038,23551,44.928000


In [8]:

train_processed = train.groupby("customer_id", as_index=False).agg(
    frequency=("customer_id", "size"),
    recency=("date", lambda x: np.round((x.max() - x.min()).days / 7)),
    T=("date", lambda x: np.round((pd.to_datetime("1998-01-01") - x.min()).days / 7)),
).assign(
    frequency=lambda x: x["frequency"] - 1
).merge(train_amount_filtered, on="customer_id")

In [9]:
train_processed

,customer_id,frequency,recency,T,amount
0,4,3,49.0,52.0,23.723333
1,21,1,2.0,52.0,11.770000
2,111,9,48.0,52.0,75.347778
3,112,1,5.0,52.0,11.770000
4,114,2,36.0,52.0,25.550000
...,...,...,...,...,...
1035,23500,8,28.0,40.0,16.192500
1036,23506,1,9.0,40.0,13.970000
1037,23507,4,38.0,40.0,30.637500
1038,23551,5,24.0,40.0,44.928000


In [10]:
customers_no_transactions_in_test = set(train["customer_id"]) - set(test["customer_id"])
customers_no_transactions_in_test

padded_test = pd.DataFrame(
    {
        "customer_id": [i for i in customers_no_transactions_in_test],
        "frequency": np.repeat(0, len(customers_no_transactions_in_test)),
        "monetary_value": np.repeat(0, len(customers_no_transactions_in_test)),
        "duration": np.round((test["date"].max() - pd.to_datetime("1998-01-01")).days / 7)
    }
)

In [11]:
test_processed = test.groupby("customer_id", as_index=False).agg(
    frequency=("customer_id", "size"),
    monetary_value=("amount", "mean")
).assign(
    duration=np.round((test["date"].max() - pd.to_datetime("1998-01-01")).days / 7),
)

test_processed = pd.concat([test_processed, padded_test])
test_processed.sort_values("customer_id")

,customer_id,frequency,monetary_value,duration
2,4,0,0.000,26.0
4,18,0,0.000,26.0
6,21,0,0.000,26.0
15,50,0,0.000,26.0
17,60,0,0.000,26.0
...,...,...,...,...
512,23537,2,19.775,26.0
1675,23551,0,0.000,26.0
513,23554,1,24.600,26.0
514,23556,2,28.980,26.0
